# Import NetCDF files from CMEMS



![step001.png](Fig-CMEMS/step001.png)



The following command download CMEMS data using the motu-client. The CMEMS portal give a script like the following matching you search criteria:

```bash
python -m  motu-client --user "abarth" --pwd  "your*password" --motu http://nrt.cmems-du.eu/motu-web/Motu \
  --service-id MEDSEA_ANALYSIS_FORECAST_PHY_006_013-TDS --product-id sv04-med-ingv-tem-an-fc-d \
  --longitude-min -17.29166603088379 --longitude-max 36.29166793823242 \
  --latitude-min 30.1875 --latitude-max 45.97916793823242 \
  --date-min "2019-04-24 00:00:00" --date-max "2019-04-24 00:00:00" \
  --depth-min 1.0181 --depth-max 104.94500000000001 \
  --variable thetao --out-dir /tmp --out-name OUTPUT_FILENAME.nc
```


Import the necessary modules for linear algebra (the transpose operator), plotting via matplotlib and reading NetCDF Files 


In [2]:
using LinearAlgebra
using PyCall
using PyCall: PyObject
using PyPlot
using NCDatasets

┌ Info: Recompiling stale cache file /home/abarth/.julia/compiled/v1.0/NCDatasets/lxvtD.ji for NCDatasets [85f8d34a-cbdd-5861-8df4-14fed0d494ab]
└ @ Base loading.jl:1190


Some helper functions to make plotting data with missing values easier

In [3]:
function PyObject(a::Array{Union{T,Missing},N}) where {T,N}
  numpy_ma = pyimport("numpy").ma
  pycall(numpy_ma.array, Any, coalesce.(a,zero(T)), mask=ismissing.(a))
end

PyObject(a::Adjoint{Array}) = PyObject(copy(a))

PyObject

Name of the downloaded NetCDF file

In [4]:
filename = "Data/sv04-med-ingv-tem-an-fc-d_1555334258448.nc"

"Data/sv04-med-ingv-tem-an-fc-d_1555334258448.nc"

List the metadata of the NetCDF file

In [5]:
ds = Dataset(filename)

NCDatasets.NetCDFError: NCDatasets.NetCDFError(2, "Opening path Data/sv04-med-ingv-tem-an-fc-d_1555334258448.nc: Datei oder Verzeichnis nicht gefunden")

The time instance in the NetCDF file

In [6]:
ds["time"][:]

UndefVarError: UndefVarError: ds not defined

The depth (in m) for the first level:

In [7]:
ds["depth"][1]

UndefVarError: UndefVarError: ds not defined

Load the first layer and the first time instance:

In [8]:
SST = ds["thetao"][:,:,1,1];

UndefVarError: UndefVarError: ds not defined

In [9]:
pcolor(SST', cmap="jet"); colorbar(orientation="horizontal");

UndefVarError: UndefVarError: SST not defined

A nicer plot with longitude and latitude as axes and the correct aspect ratio:

In [10]:
lon = ds["lon"][:];
lat = ds["lat"][:];
pcolor(lon,lat,SST', cmap="jet"); colorbar(orientation="horizontal");
gca().set_aspect(1/cos(pi*mean(lat) / 180))

UndefVarError: UndefVarError: ds not defined

List a depth levels in the NetCDF file

In [11]:
depth = ds["depth"][:]

UndefVarError: UndefVarError: ds not defined

Plot a vertical section at a constant latitude:

In [12]:
j = 200;
T = ds["thetao"][:,j,:,1];
lat[j]

UndefVarError: UndefVarError: ds not defined

In [13]:
pcolor(lon,-depth,T'); colorbar(orientation="horizontal");

UndefVarError: UndefVarError: depth not defined

# In situ observations

In [14]:
using PhysOcean

┌ Info: Recompiling stale cache file /home/abarth/.julia/compiled/v1.0/PhysOcean/9Hv8Y.ji for PhysOcean [3725be50-bbbd-5592-92c3-2f0e82159c3e]
└ @ Base loading.jl:1190


In [15]:
# username = "Joe"
# password = "p*ssw0rd"

username, password = split(read(expanduser("~/.cmems-credentials"),String),":", limit = 2);

In [16]:
?CMEMS.download

```
CMEMS.download(lonr,latr,timerange,param,username,password,basedir[; indexURLs = ...])
```

Download in situ data within the longitude range `lonr` (an array or tuple with two elements: the minimum longitude and the maximum longitude), the latitude range `latr` (likewise), time range `timerange` (an array or tuple with two `DateTime` structures: the starting date and the end date) from the CMEMS (Copernicus Marine environment monitoring service) in situ service [^1]. `param` is one of the parameter codes as defined in [^2] or [^3]. `username` and `password` are the credentials to access data [^1] and `basedir` is the directory under which the data is saved. `indexURLs` is a list of the URL to the `index_history.txt` file. Per default, it includes the URLs of the Baltic, Arctic, North West Shelf, Iberian, Mediteranean and Black Sea Thematic Assembly Center.

As these URLs might change, the latest version of the URLs to the indexes can be obtained at [^1].

# Example

```julia-repl
julia> username = "..."
julia> password = "..."
julia> lonr = [7.6, 12.2]
julia> latr = [42, 44.5]
julia> timerange = [DateTime(2016,5,1),DateTime(2016,8,1)]
julia> param = "TEMP"
julia> basedir = "/tmp"
julia> files = CMEMS.download(lonr,latr,timerange,param,username,password,basedir)
```

[^1]: http://marine.copernicus.eu/

[^2]: http://www.coriolis.eu.org/Documentation/General-Informations-on-Data/Codes-Tables

[^3]: http://doi.org/10.13155/40846


In [17]:
lonr = [-Inf, Inf]
latr = [-Inf, Inf]
timerange = [DateTime(2018,1,1),DateTime(2018,1,10)]
param = "TEMP"
basedir = "/tmp"
files = CMEMS.download(lonr,latr,timerange,param,username,password,basedir; 
    indexURLs = ["ftp://my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/index_history.txt"])

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1926k  100 1926k    0     0  1285k      0  0:00:01  0:00:01 --:--:-- 1284k


362-element Array{String,1}:
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_1300529.nc"        
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_2500622.nc"        
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_2601561.nc"        
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_4100668.nc"        
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_4400510.nc"        
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_4400513.nc"        
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_4400765.nc"        
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_4400886.nc"        
 "my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/drifter/GL_TS_DB_4400899.nc"        
 "my.cmems-du.eu/

In [18]:
cd(basedir)

In [19]:
obsdata,obslon,obslat,obsz,obstime,obsids = CMEMS.load(Float64,files,"TEMP");

ErrorException: unexpected size in my.cmems-du.eu/Core/INSITU_NWS_TS_REP_OBSERVATIONS_013_043/history/mooring/NO_TS_MO_62029.nc, size(data) = (2, 201920), size(lon) = (1,)